In [1]:
!pip install yfinance exa_py anthropic

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 222.8/222.8 kB 5.9 MB/s eta 0:00:00


In [14]:
import re
import json
import requests
import yfinance as yf
from exa_py import Exa
import anthropic
from google.colab import userdata

# ----------------------
# Configuration / API Keys
# ----------------------
ANTHROPIC_API_KEY = userdata.get('ANTHROPIC_API_KEY')
PERPLEXITY_API_KEY = userdata.get('PERPLEXITY_API_KEY')
EXA_API_KEY = userdata.get('EXA_API_KEY')


MODEL_NAME = "claude-3-5-sonnet-20241022"
client = anthropic.Anthropic(api_key=ANTHROPIC_API_KEY)
exa = Exa(api_key=EXA_API_KEY)

def fetch_yahoo_finance(ticker):
    """Fetches last week's market data for a given ticker using yfinance."""
    try:
        stock = yf.Ticker(ticker)
        hist = stock.history(period="7d")
        if hist.empty:
            return f"Yahoo Finance: No market data found for {ticker} in the last week."
        return f"Yahoo Finance: Last week's market data for {ticker}:\n{hist.to_string()}"
    except Exception as e:
        return f"Error fetching Yahoo Finance data: {e}"

def fetch_perplexity_news(ticker):
    """Calls the Perplexity Chat Completions API to fetch recent news headlines for a ticker."""
    url = "https://api.perplexity.ai/chat/completions"
    headers = {
         "Authorization": f"Bearer {PERPLEXITY_API_KEY}",
         "Content-Type": "application/json"
    }
    payload = {
         "model": "sonar",
         "messages": [
              {"role": "system", "content": "Be precise and concise."},
              {"role": "user", "content": f"Provide the latest news headlines and summaries for {ticker}."}
         ],
         "max_tokens": 500,
         "temperature": 0.2,
         "top_p": 0.9,
         "top_k": 0,
         "stream": False,
         "presence_penalty": 0,
         "frequency_penalty": 1,
         "search_recency_filter": "week"
    }
    try:
        response = requests.post(url, json=payload, headers=headers)
        if response.status_code == 200:
            data = response.json()
            answer = data.get("choices", [{}])[0].get("message", {}).get("content", "")
            return f"Perplexity News: {answer}"
        else:
            return f"Error fetching Perplexity news: {response.text}"
    except Exception as e:
        return f"Exception in Perplexity API call: {e}"

def fetch_exa_search(ticker, start_date, end_date, num_results=5):
    try:
        query = f"financial news and reports on {ticker}"
        result = exa.search_and_contents(
            query,
            text=True,
            num_results=num_results,
            start_published_date=start_date,
            end_published_date=end_date,
            include_text=[ticker]
        )
        results = result.results
        if results:
            entries = []
            for res in results:
                title = getattr(res, "title", None)
                if not title:
                    title = getattr(res, "url", "No title")
                content = getattr(res, "text", None)
                if not content:
                    content = getattr(res, "text", "No content available.")
                published_date = getattr(res, "published_date", "N/A")
                entry = (
                    f"Title: {title}\n"
                    f"Published: {published_date}\n"
                    f"Content: {content}"
                )
                entries.append(entry)
            return f"Exa Search: Found {len(entries)} results:\n" + "\n\n".join(entries)
        else:
            return f"Exa Search: No results found for {ticker} in the given period."
    except Exception as e:
        return f"Error fetching Exa Search results: {e}"

def calculate(expression):
    """Evaluates a sanitized arithmetic expression."""
    expression = re.sub(r'[^0-9+\-*/().]', '', expression)
    try:
         result = eval(expression)
         return str(result)
    except (SyntaxError, ZeroDivisionError, NameError, TypeError, OverflowError):
         return "Error: Invalid expression"

tools = [
    {
        "name": "yahoo_finance",
        "description": (
            "Fetches current stock data for a given ticker from Yahoo Finance. "
            "Provides real-time pricing, volume, and market trends. "
            "Use this tool when you need quantitative data about a stock."
        ),
        "input_schema": {
            "type": "object",
            "properties": {
                "ticker": {
                    "type": "string",
                    "description": "The stock ticker symbol (e.g., TSLA for Tesla, Inc.)."
                }
            },
            "required": ["ticker"]
        }
    },
    {
        "name": "perplexity_news",
        "description": (
            "Fetches recent news and financial reports for a given stock ticker using the Perplexity API. "
            "Use this tool to retrieve the latest headlines and summaries related to the stock."
        ),
        "input_schema": {
            "type": "object",
            "properties": {
                "ticker": {
                    "type": "string",
                    "description": "The stock ticker symbol (e.g., TSLA)."
                }
            },
            "required": ["ticker"]
        }
    },
    {
        "name": "exa_search",
        "description": (
            "Performs a detailed search for financial news and reports on a given stock ticker using the Exa AI Search API. "
            "Use this tool when you need to fetch comprehensive search results, including reports within a specific date range."
        ),
        "input_schema": {
            "type": "object",
            "properties": {
                "ticker": {
                    "type": "string",
                    "description": "The stock ticker symbol (e.g., TSLA)."
                },
                "start_date": {
                    "type": "string",
                    "description": "The start date for the search in ISO format (e.g., '2024-11-15T08:00:00.000Z')."
                },
                "end_date": {
                    "type": "string",
                    "description": "The end date for the search in ISO format (e.g., '2025-02-16T07:59:59.999Z')."
                },
                "num_results": {
                    "type": "integer",
                    "description": "The number of search results to return.",
                    "default": 5
                }
            },
            "required": ["ticker", "start_date", "end_date"]
        }
    },
    {
        "name": "calculator",
        "description": (
            "A simple calculator that performs basic arithmetic operations. "
            "Use this tool to perform any necessary calculations (e.g., computing ratios or adjusting numbers)."
        ),
        "input_schema": {
            "type": "object",
            "properties": {
                "expression": {
                    "type": "string",
                    "description": "The mathematical expression to evaluate (e.g., '2 + 3 * 4')."
                }
            },
            "required": ["expression"]
        }
    }
]

def process_tool_call(tool_name, tool_input):
    if tool_name == "yahoo_finance":
        return fetch_yahoo_finance(tool_input["ticker"])
    elif tool_name == "perplexity_news":
        return fetch_perplexity_news(tool_input["ticker"])
    elif tool_name == "exa_search":
        ticker = tool_input["ticker"]
        start_date = tool_input["start_date"]
        end_date = tool_input["end_date"]
        num_results = tool_input.get("num_results", 5)
        return fetch_exa_search(ticker, start_date, end_date, num_results)
    elif tool_name == "calculator":
        return calculate(tool_input["expression"])
    else:
        return f"Error: Unknown tool {tool_name}"

def main():
    ticker = input("Enter the stock ticker (e.g., TSLA): ").strip().upper()
    start_date ="2024-1-15" # input("Enter start date for news search (ISO format, e.g., 2024-11-15T08:00:00.000Z): ").strip()
    end_date = "2025-2-15" # input("Enter end date for news search (ISO format, e.g., 2025-02-16T07:59:59.999Z): ").strip()

    conversation = [
        {
            "role": "user",
            "content": (
                f"Please provide a comprehensive investment assessment for {ticker}. "
                "Include an analysis of the current stock data, recent news, and any additional financial reports if available. "
                "Use the available tools as needed: use 'yahoo_finance' for current market data, 'perplexity_news' for recent news, "
                "'exa_search' for a detailed search over a specified date range, and 'calculator' for any necessary arithmetic."
                "Please make sure to use the calculator at least two times to generate accurate numerical results."
            )
        }
    ]
    system_prompt=(
    "You are an investment analysis assistant. Your task is to provide a comprehensive "
                "investment assessment for a given stock. Use the following tools as needed:\n\n"
                "1. yahoo_finance: To get real-time stock data (price, volume, trends).\n"
                "2. perplexity_news: To fetch recent news headlines and financial reports about the stock.\n"
                "3. exa_search: To perform an in-depth search for financial news and detailed reports within a specified date range.\n"
                "4. calculator: To perform any arithmetic calculations required during your analysis.\n\n"
                "Make sure to use the tools only when needed and incorporate their results into your final investment recommendation."
                "When you are done stop using tools and just provide your final verdict!!!")

    while True:
        response = client.messages.create(
            model=MODEL_NAME,
            system=system_prompt,
            max_tokens=4096,
            tools=tools,
            messages=conversation
        )

        conversation.append({
            "role": "assistant",
            "content": response.content
        })

        tool_called = False
        for block in response.content:
            if block.type == "tool_use":
                tool_called = True
                tool_name = block.name
                tool_input = block.input
                print(f"\n[Agent requests tool call] {tool_name} with input: {json.dumps(tool_input)}")
                result = process_tool_call(tool_name, tool_input)
                print(f"[Tool '{tool_name}' result] {result}\n")

                tool_result_message = {
                    "role": "user",
                    "content": [
                        {
                            "type": "tool_result",
                            "tool_use_id": block.id,
                            "content": result
                        }
                    ]
                }
                conversation.append(tool_result_message)
                break

        if not tool_called:
            final_answer = ""
            for block in response.content:
                if block.type == "text":
                    final_answer += block.text
            print("\n=== Final Investment Assessment ===")
            print(final_answer)
            break

if __name__ == "__main__":
    main()


Enter the stock ticker (e.g., TSLA): CSCO

[Agent requests tool call] yahoo_finance with input: {"ticker": "CSCO"}
[Tool 'yahoo_finance' result] Yahoo Finance: Last week's market data for CSCO:
                                Open       High        Low      Close    Volume  Dividends  Stock Splits
Date                                                                                                    
2025-02-06 00:00:00-05:00  62.820000  62.910000  62.070000  62.270000  15521800        0.0           0.0
2025-02-07 00:00:00-05:00  62.740002  63.250000  62.209999  62.270000  23861000        0.0           0.0
2025-02-10 00:00:00-05:00  63.119999  63.150002  62.380001  62.810001  15816200        0.0           0.0
2025-02-11 00:00:00-05:00  62.529999  62.799999  62.180000  62.430000  17702500        0.0           0.0
2025-02-12 00:00:00-05:00  61.439999  62.560001  61.200001  62.529999  30116300        0.0           0.0
2025-02-13 00:00:00-05:00  66.099998  66.500000  62.720001  63.840000  